In [1]:
from algorithms import CDAE_sparse,CDAE
import tensorflow as tf
import pandas as pd
import numpy as np
import os,sys
from scipy.sparse import *
from numpy.random import choice


In [2]:
def load_data(csv_file, shape):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['uid']), np.array(tp['sid'])
    seq = np.concatenate((rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int') ), axis=1)
    data = csr_matrix((np.ones_like(rows), (rows, cols)), dtype=np.float32, shape=shape)
    return data, seq

In [58]:
DATA_DIR = 'data/ml-1m/pro'
uid_fname = 'unique_uid.txt'
sid_fname = 'unique_sid.txt'
rating_fname = 'train.csv'
test_fname ='test.csv'
unique_uid = list()
with open(os.path.join(DATA_DIR, uid_fname), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
unique_sid = list()

with open(os.path.join(DATA_DIR, sid_fname), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())
n_items = len(unique_sid)
n_users = len(unique_uid)

train_data, train_raw = load_data(os.path.join(DATA_DIR, rating_fname),(n_users,n_items))
test_data, test_raw = load_data(os.path.join(DATA_DIR, test_fname),(n_users,n_items))
interaction_data = train_data + test_data

In [59]:
l = interaction_data.shape[0]
train_data = interaction_data[:int(0.8*l)]
test_data = interaction_data[int(0.8*l):].todense()
temp = test_data.copy()
hidden = np.multiply(temp ,(np.random.random(size = temp.shape) >= 0.5))
visible = temp - hidden

In [60]:
vec = []
for i in range(train_data.shape[0]):
    vec.append(train_data[i].nonzero()[1])

In [61]:
negs = []
for i in range(len(vec)*200):
    negs.append(choice(n_items,500))

In [62]:
hidden.shape,visible.shape

((1207, 3502), (1207, 3502))

In [63]:
import importlib
importlib.reload(CDAE_sparse)
model = CDAE_sparse.CDAE(n_users, n_items, latent_dim =200, learning_rate = 0.15,
                        keep_prob = 0.4, activation = 'identity', use_user_bias = False,lambda_total = 0.0)

In [64]:
model.train_model(train_data,n_iter =50,dropout_rate = 0.8, 
                  test_visible = visible,test_hidden = hidden, k = 10)

INFO:tensorflow:Scale of 0 disables regularizer.
[iteration 0]	[precision@10 0.243]	[recall@10 0.069]
[iteration 1]	[precision@10 0.250]	[recall@10 0.072]
[iteration 2]	[precision@10 0.236]	[recall@10 0.067]
[iteration 3]	[precision@10 0.247]	[recall@10 0.072]
[iteration 4]	[precision@10 0.269]	[recall@10 0.074]
[iteration 5]	[precision@10 0.296]	[recall@10 0.083]
[iteration 6]	[precision@10 0.307]	[recall@10 0.086]
[iteration 7]	[precision@10 0.328]	[recall@10 0.095]
[iteration 8]	[precision@10 0.330]	[recall@10 0.095]
[iteration 9]	[precision@10 0.329]	[recall@10 0.092]
[iteration 10]	[precision@10 0.339]	[recall@10 0.097]
[iteration 11]	[precision@10 0.337]	[recall@10 0.096]
[iteration 12]	[precision@10 0.343]	[recall@10 0.097]
[iteration 13]	[precision@10 0.355]	[recall@10 0.102]
[iteration 14]	[precision@10 0.354]	[recall@10 0.104]
[iteration 15]	[precision@10 0.357]	[recall@10 0.106]
[iteration 16]	[precision@10 0.357]	[recall@10 0.106]
[iteration 17]	[precision@10 0.364]	[recall